In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, avg, count, min, max

# Create Spark Session
spark = SparkSession.builder \
    .appName("Aggregation Lab") \
    .master("local[*]") \
    .getOrCreate()

# Sample data
sales_data = [
    ("Alice", "Electronics", 300),
    ("Bob", "Clothing", 100),
    ("Alice", "Clothing", 150),
    ("Bob", "Electronics", 200),
    ("Charlie", "Clothing", 50),
    ("Charlie", "Electronics", 400),
]

# Define schema
columns = ["name", "category", "amount"]

# Create DataFrame
df = spark.createDataFrame(sales_data, columns)

# Show raw data
df.show()

# Register as SQL table
df.createOrReplaceTempView("sales")
# Total sales
df.groupBy().sum("amount").show()

# Average sale amount
df.groupBy().avg("amount").show()

# Total per category
df.groupBy("category").sum("amount").show()

# Count purchases per customer
df.groupBy("name").count().show()

# Max spend by category
df.groupBy("category").max("amount").show()
# Total sales using SQL
spark.sql("SELECT SUM(amount) as total_sales FROM sales").show()

# Average sale amount
spark.sql("SELECT AVG(amount) as avg_amount FROM sales").show()

# Total per category
spark.sql("SELECT category, SUM(amount) as total FROM sales GROUP BY category").show()

# Count of transactions per user
spark.sql("SELECT name, COUNT(*) as tx_count FROM sales GROUP BY name").show()

# Min spend by category
spark.sql("SELECT category, MIN(amount) as min_amount FROM sales GROUP BY category").show()
# Who spent more than 400 in total?
df.groupBy("name").sum("amount").filter(col("sum(amount)") > 400).show()

# SQL equivalent
spark.sql("""
SELECT name, SUM(amount) as total 
FROM sales 
GROUP BY name 
HAVING total > 400
""").show()

+-------+-----------+------+
|   name|   category|amount|
+-------+-----------+------+
|  Alice|Electronics|   300|
|    Bob|   Clothing|   100|
|  Alice|   Clothing|   150|
|    Bob|Electronics|   200|
|Charlie|   Clothing|    50|
|Charlie|Electronics|   400|
+-------+-----------+------+

+-----------+
|sum(amount)|
+-----------+
|       1200|
+-----------+

+-----------+
|avg(amount)|
+-----------+
|      200.0|
+-----------+

+-----------+-----------+
|   category|sum(amount)|
+-----------+-----------+
|Electronics|        900|
|   Clothing|        300|
+-----------+-----------+

+-------+-----+
|   name|count|
+-------+-----+
|  Alice|    2|
|    Bob|    2|
|Charlie|    2|
+-------+-----+

+-----------+-----------+
|   category|max(amount)|
+-----------+-----------+
|Electronics|        400|
|   Clothing|        150|
+-----------+-----------+

+-----------+
|total_sales|
+-----------+
|       1200|
+-----------+

+----------+
|avg_amount|
+----------+
|     200.0|
+----------+

+--